- 세그먼트 : 이미지를 여러개의 영역으로 나누는 작업으로 컴퓨터의 비전 및 이미지 처리에 사용
  - 이진 세그먼트
  - 다중 클래스 세그먼트
  - 인스턴스 세그먼트
    - 같은 클래스의 객체를 개별적으로 구분
  - 파노라믹 세그먼트
    - 인스턴스 세그먼트와 픽셀 분류의 조합, 모든 픽셀을 객체로 할당하거나 배경으로 분류

참고 자료 : https://www.tensorflow.org/tutorials/images/segmentation?hl=ko

In [47]:
!curl -O http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!curl -O http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
!tar -xf images.tar.gz
!tar -xf annotations.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   269  100   269    0     0    284      0 --:--:-- --:--:-- --:--:--   284
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   274  100   274    0     0    772      0 --:--:-- --:--:-- --:--:--   774
tar: This does not look like a tar archive

gzip: stdin: not in gzip format
tar: Child returned status 1
tar: Error is not recoverable: exiting now
tar: This does not look like a tar archive

gzip: stdin: not in gzip format
tar: Child returned status 1
tar: Error is not recoverable: exiting now
